# Chapter 1. Introduction to functional programming

## 1.1 What is this thing called functional programming?

At a very high level, it is a programming style that emphasises on
1. Functions as first-class values
2. Avoiding state mutation

### 1.1.1 Functions as first-class values

* allows function to be used as input and output of other functions
* assign function to variables - in other words, you can treat it like a value

treating function as value allow us to write very concise and powerful code

In [ ]:
Func<int, int> triple = x => x * 3;
var range = Enumerable.Range(1, 3);
var triples = range.Select(triple);
triples

index,value
0,3
1,6
2,9


### 1.1.2 Avoiding state mutation

When following functional paradigm, we should refrain from state mutation.

* once created, an object `never` changes and variables should `never` be reassigned

`muataion` means that a value is changed in-place. It is also referred to as destructive updates


In [ ]:
var nums = new List<int>{1, 2, 3 };
nums[2] = 7;

nums // original list is mutated

index,value
0,1
1,2
2,7


In [ ]:
// function / operation that does mutaion
var nums = new List<int>{3, 2, 1};
nums.Sort();
nums


index,value
0,1
1,2
2,3


In [ ]:

// functional version - no mutation
var nums1 = new List<int>{3, 2, 1};
var result = nums1.OrderBy(x => x);
(nums1, result)

Item1,Item2
"[ 3, 2, 1 ]","[ 1, 2, 3 ]"


### 1.1.3 Writing programs with strong grarantees



In [ ]:
// Example of why state mutation is bad

using static System.Linq.Enumerable;
using static System.Console;

var list = Range(-10000, 20001).ToList();
Action task1 = () => {WriteLine(list.Sum());};
Action task2 = () => {list.Sort(); WriteLine(list.Sum());};
Parallel.Invoke(task1, task2);
// expect 0 from both tasks as result but due to in-place mutation, we get wrong result


37


0


In [ ]:

// Example without state mutation

using static System.Linq.Enumerable;
using static System.Console;

var list = Range(-10000, 20001).ToList();
Action task1 = () => {WriteLine(list.Sum());};
Action task2 = () => {list.OrderBy( x => x); WriteLine(list.Sum());};
Parallel.Invoke(task1, task2);

0


0


> "Although most people will agree that treating functions as first-class values and avoiding state mutation are fundamental tenets of FP, their application gives rise to a series of practices and techniques, so it’s debatable which techniques should be considered essential and included in a book like this. I encourage you to take a pragmatic approach to the subject and try to understand FP as a set of tools that you can use to address your programming tasks. As you learn these techniques, you’ll start to look at problems from a different perspective: you’ll start to think functionally"

#### Functional vs. object-oriented?

Fundamental priciples of OOP (encapsuation, data abstration, and so on) are `orthogonal` to the priciples of FP, so there's no reason why the two paradigms can't be combined.

The real question is `imperative vs functional`

The idea of structuring a large and complex appication relies on serveral principles
* Modularity
* Separation of concerns
* Layering
* Loose coupling

These can be acheived in both OOP and FP - the mechanics of acheiving them might be different

## 1.2 How functional a language is C#?

* Functions are first-class values in C#
* GC helps with immutable style by collecting discarded old objects
* C# is not discouraging in-place updates so it is not so great on this aspect

In summary, some aspects are supported very well but not others

### 1.2.1 The functional nature of LINQ

Although, it is not apparent at first glance, LINQ is functional library. By default, it supports operations on `IEnumerable`.

The most common operations are:
* mapping
* sorting
* filtering

the following is an example of this

In [ ]:
var result = Enumerable.Range(1, 100)
.Where(i => i % 20 == 0)
.OrderBy(i => -i)
.Select(i => $"{i}%");
result

index,value
0,100%
1,80%
2,60%
3,40%
4,20%


Commonly, C# developers stick to functional style when using types that support LINQ and other times they stick to imperative style. So, most of the C# code base are patch work of functional and imperative style.

This means, C# developers are aware of the benefits of FP but they don't have enough exposure to the design principles behind LINQ to leverage those techiniques in their own designs.

### 1.2.2 Functional features in C# 6 and C# 7

* importing static members with `using static`
* easier immutable types with getter-only auto properies
* more concise functions with expression-bodied members
* local functions
* better syntax for tuples


### 1.2.3 A more functional future for C#?

* record types
* algebraic data types
* pattern matching
* better syntax for tuples


## 1.3 Thinking in functions

### 1.3.1 Functions as map
In mathematics, function is a map between two sets, respectively called the domain and codomain.

In programming, you can think of domain and codomain as data types.

### 1.3.2 Representing functions in C#

* methods
* delegates
* lambda expressions
* dictionaries


#### Function arity
`arity` is a funny word that refers to the number of arguments that a function accepts.
* `nullary` function takes no arguments
* `unary` function takes one arguments
* `binary` fuction takes two arguments
* `ternary` for 3

> Note: one difference between normal fuction and lambda expression is - `closure`



## 1.4 Higher-order functions

`HOF`s are functions that take other functions as input or return a function as output, or both.

### 1.4.1 Functions that depend on other functions

Some HOFs take other functions as arguments and invoke them in order to do their work.

`Where` and `Sort` methods are couple of examples. They apply passed in function on each items in `IEnumerable`. These are also examples of `iterated applications`

The general scheme for this look like this, where `f` is passed from outside

```
IterativelyApply(f, ...)
{
    for (...)
        f(...)
}
```

Another popular HOF scheme is optional execution
```
ConditionallyApply(f, ...)
{
    if(...)
        f(...)
}
```

Here is a realistic example of optional execution HOF

```cs
class Cache<T> where T: class
{
    public T Get(Guid id) => // ...
    
    public T Get(Guid id, Func<T> onMiss)
        => Get(id) ?? onMiss();
}
```

passing another function as input is sometimes referred to as `inversion of control`: the caller of the HOF decides what to do by supplying functions and the callee decides when to do it by invoking the given function.

### 1.4.2 Adapter functions

Some `HOF`s don't apply the given function at all, but rather return a new function, somehow related to the function given as an argument.

Example: returns a new function with swapped arguments
```cs
static Func<T2, T1, R> SwapArgs<T1, T2, R>(this Func<T1, T2, R> f)
    => (t2, t1) => f(t1, t2);
```

### 1.4.3 Functions that create other functions

Sometimes you will create fuctions whose primary purpose is to create other functions - you can think of them as `function factories`.

Example: returns a fuction with given mod number
```cs
Func<int, bool> isMod(int n) => i => i % n == 0;
```

Usage:
```cs
Range(1, 20).Where(isMod(2));
Range(1, 20).Where(isMod(3));
```
We get not only generality but also readability

## 1.5 Using HOFs to avoid duplication

Another common use case for `HOF`s is to encapsulate setup and teardown operations.

Example of setup and teardown:
```cs
using (var conn = new SqlConnection(connString))
{
    conn.Open();
    // interact with database
}
```
### 1.5.1 Encapsulating setup and teardown into a HOF

Here is the scheme for this HOF
```
SetupTeardown(f, ...)
{
    Setup()
    f(...)
    Teardown()
}
```
Sometimes this is referred to as `hole in the middle`

Here is an example of database setup and teardown encapsulated in HOF

```cs
public static class ConnectionHelper
{
    public static R Connect<R>(string connString, Func<IDbConnection, R> f)
    {
        using(var conn = new SqlConnection(connString))
        {
            conn.Open();
            return f(conn);
        }
    }
}
```

Usage: 
```cs
public void Log(LogMessage message) 
    => Connect(connString, c => c.Execute("sp_create_log", message, commandType: commandType.StoredProcedure));
```

### 1.5.2 Turning the using statement into HOF

```cs
public static R Using<TDisp, R>(TDisp disposable, Func<TDisp, R> f) where TDisp: IDisposable
{
    using(disposable) return f(disposable);
}
```

with this `Connect` method can be rewritten as follow:

```cs
public static R Connect<R>(string connString, Func<IDbConnection, R> f)
    => Using(new SqlConnection(connString)), conn => {conn.Open(); return f(conn);});
```

### 1.5.3 Tradeoffs of HOFs

* Conciseness
* Avoid duplication
* Separation of concerns

Drawbacks:
* increased stack use
* debugging is bit more complex due to callbacks


## 1.6. Benefits of functional programming

* Cleaner code
* Better support for concurrency
* A multi-paradigm approach (if you only have hammer, every problem would look like a nail)